In [3]:
import pandas as pd
import numpy

In [5]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}

In [6]:
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [8]:
datas = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=50000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [9]:
list(datas.columns)

['資料日期與時間',
 '餐別帶',
 '縣市別',
 '店舖代號',
 '主商圈',
 '品號-品名稱',
 '群號-群名稱',
 '單品名稱',
 '銷售數量',
 '銷售單價',
 '交易金額']

In [43]:
def get_transaction_dict(df):
    transaction_df = df.filter(['交易id'])
    groupbyObject = transaction_df.groupby(['交易id'])
    return groupbyObject.first()

In [11]:
def get_items_dict(df):
    items = df.set_index('單品名稱')
    items = items.drop(['餐別帶', '資料日期與時間', '縣市別', '店舖代號', '主商圈', '銷售數量', '交易金額'], axis=1)
    return items.groupby(items.index).first().to_dict('index')

In [ ]:
class TransactionTransformer:
    def __init__(self, transaction_id_name, item_name, transaction_amount_name, transaction_attrs=[], item_attrs=[]):
        self.transaction_id_name = transaction_id_name
        self.item_name = item_name
        self.transaction_attrs = transaction_attrs
        self.item_attrs = item_attrs
        self.transaction_amount_name = transaction_amount_name

    def get_transaction_dict(self, df):
        total_cols = list(df.columns)
        transaction_df = df.filter(self.transaction_attrs + [self.transaction_id_name, self.transaction_amount_name])
        groupbyObject = transaction_df.groupby([self.transaction_id_name])
        transaction_df = groupByObject.agg({ key: 'first' if key != self.transaction_amount_name else 'sum' for key in self.transaction_attrs })
        return transaction_df.to_dict('index')
    
    def get_transaction_length(self, df):
        return len(df.index.unique())

    def get_nodes(self, df):
        item_df = df.set_index(self.item_name)
        item_df = item_df.filter(self.item_attrs)
        item_dict = items.groupby(items.index).first().to_dict('index')
        for key, value in item_dict.items():
            value[self.item_name] = key
        return item_dict
    
    def get_edges(self, df, nodes, support, weight_func):
        df = df.filter([self.item_name, self.transaction_id_name])
        ts_dict = 
        for index, row in df:
            
    def get_copurchase_df(self, df):
        count_size = df.index.unique()
        size_df = df.groupby(self.transaction_id_name).size()
        size_df = size_df[size_df < 2]
        return df.drop(list(size_df.index), axis=0)

    def get_graph_view(self, df, support, weight_func):
        df = df.dropna()
        copurchase_df = self.get_copurchase_df(df)
        nodes = self.get_nodes(df)
#         edges = 